In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import GridSearchCV, KFold
from IPython.core.debugger import set_trace
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Data Collection

In [ ]:
# Gradient Descent

In [27]:
# Cost Function

def cost_fn(x, y, w):                                                   
    z = x@w.T
    print((y@z.T).diagonal())
    print(z.max(axis=1))
    J = -((y@z.T).diagonal() - (z.max(axis=1) + np.log(np.exp(z - z.max(axis=1, keepdims=True)).sum(axis=1)))).mean()
    return J

In [ ]:
# Softmax Classifier

class softmax_classifier:
    def __init__(self, add_bias=True, batch_size=10, lerning_rate=0.1, momentum=0.1, epsilon=1e-4, max_iters=1e5, verbose=False):
        self.add_bias = add_bias
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.epsilon = epsilon
        self.max_iters = max_iters
        self.verbose = verbose
        

In [ ]:
# Hyperparameter Optimization

In [ ]:
# Model for comparison